In [56]:
!pip install textaugment &> /dev/null
!pip install torch &> /dev/null
!pip install nlpaug &> /dev/null
!pip install wget &> /dev/null
!pip install torchtext &> /dev/null
!pip install --no-cache-dir transformers sentencepiece &> /dev/null


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [3]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eishmaheshwari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/eishmaheshwari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/eishmaheshwari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/eishmaheshwari/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [48]:
import gzip
import shutil
import os
import wget

gn_vec_path = "GoogleNews-vectors-negative300.bin"
if not os.path.exists("GoogleNews-vectors-negative300.bin"):
    if not os.path.exists("../Ch3/GoogleNews-vectors-negative300.bin"):
        # Downloading the reqired model
        if not os.path.exists("../Ch3/GoogleNews-vectors-negative300.bin.gz"):
            if not os.path.exists("GoogleNews-vectors-negative300.bin.gz"):
                wget.download("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz")
            gn_vec_zip_path = "GoogleNews-vectors-negative300.bin.gz"
        else:
            gn_vec_zip_path = "../Ch3/GoogleNews-vectors-negative300.bin.gz"
        # Extracting the required model
        with gzip.open(gn_vec_zip_path, 'rb') as f_in:
            with open(gn_vec_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    else:
        gn_vec_path = "../Ch3/" + gn_vec_path

print(f"Model at {gn_vec_path}")

  8% [...                                             ]  134389760 / 1647046227

KeyboardInterrupt: 

In [51]:
import torchtext.vocab as vocab
import numpy as np
import requests
import zipfile
import io

# Download class resources...
r = requests.get("http://web.stanford.edu/class/cs21si/resources/unit1_resources.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

VEC_SIZE = 300
glove = vocab.GloVe(name='6B', dim=VEC_SIZE)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                             
100%|███████████████████████████████▉| 399999/400000 [00:34<00:00, 11567.78it/s]


In [41]:
from textaugment import EDA
import random

class DataAug:
    def __init__(self):
        self.eda = EDA()
        
    def synonym_replacement(self, text):
        return self.eda.synonym_replacement(text)
    
    def random_deletion(self, text, p=0.2):
        return self.eda.random_deletion(text, p)
    
    def random_swap(self, text):
        return self.eda.random_swap(text)
    
    def random_insertion(self, text):
        return self.eda.random_insertion(text)
    
    def shuffle_sentences(self, text):
        sentences = nltk.sent_tokenize(text)
        random.shuffle(sentences)
        return ' '.join(sentences)
    
    def contextual_embedding(self, text):
        # BERT
        aug = naw.ContextualWordEmbsAug(
            model_path='bert-base-uncased', action="insert")
        return aug.augment(text)
    
    def noncontextual_embeddings(self, text):
        # word2vec
        naw.WordEmbsAug(model_type='word2vec', model_path='.', action="insert")
        return aug.augment(text)

    


In [42]:
print(DataAug().noncontextual_embeddings("Hi there. My name is Dr. Eish and my best friend is Jeff. He is very sus"))

IsADirectoryError: [Errno 21] Is a directory: '.'

In [27]:
import nlpaug.augmenter.word as naw
import torch



Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

the fat quick brown farm fox had jumped over the lazy dog


In [53]:
import pandas as pd
import os

In [54]:
d = pd.read_csv("data/twtc.csv")
scouting_data = d[['text', 'label']]
print(scouting_data.shape)
scouting_data.head()

(9175, 2)


,text,label
0,PERSON is a Level 1 sex offender and wouldn't ...,0
1,The way teams value PERSON depends on whether ...,0
2,PERSON PERSON was the headliner in the three-p...,-1
3,PERSON made headlines for all the wrong reason...,-1
4,"The ORGANIZATION have acquired PERSON twice, ...",1


In [55]:
print("Not selected", scouting_data[scouting_data["label"] == 0].shape[0])
print("Selected", scouting_data[scouting_data["label"] == 1].shape[0])
print("No data", scouting_data[scouting_data["label"] == -1].shape[0])

Not selected 5664
Selected 2114
No data 1397


In [38]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(scouting_data, test_size=0.2)

In [60]:
unlabeled = scouting_data[scouting_data["label"] == -1]
unlabeled.to_csv('unlabaled_scouting.csv', index=False)

In [62]:
labeled = scouting_data[scouting_data["label"] >= 0]
print(labeled)
labeled.to_csv('labeled_scouting.csv', index=False)

                                                   text  label
0     PERSON is a Level 1 sex offender and wouldn't ...      0
1     The way teams value PERSON depends on whether ...      0
4      The ORGANIZATION have acquired PERSON twice, ...      1
5     Signed for an above-slot $2 million as a ORGAN...      1
6      It often takes time for those high-ceilinged,...      1
...                                                 ...    ...
9170  PERSON set a Rice record with 14 saves as soph...      0
9171  PERSON set the PERSON record for saves last ye...      0
9172  As a sophomore, in 2013, PERSON set the ORGANI...      0
9173   Another prospect in the ORGANIZATION haul, PE...      1
9174  A year after reaching Double-A for the first t...      1

[7778 rows x 2 columns]
